OPENAI_API_KEY is ready


Youtube Transcript Loader
https://llamahub.ai/

Flask App with React Frontend

https://gpt-index.readthedocs.io/en/latest/guides/tutorials/fullstack_app_guide.html
https://github.com/logan-markewich/llama_index_starter_pack/tree/main/flask_react


In [1]:
from llama_index import LLMPredictor, VectorStoreIndex, ServiceContext
from langchain import OpenAI

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model='gpt-3.5-turbo'))


In [3]:

# configure service context
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# build index
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context)

NameError: name 'documents' is not defined

In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.3/476.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: typing-inspect
    Found existing installation: typing-inspect 0.9.0
    Uninstalling typing-inspect-0.9.0:
      Successfully uninstalled typing-inspect-0.9.0
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


Design Philosophy: Progressive Disclosure of Complexity
Progressive disclosure of complexity is a design philosophy that aims to strike a balance between the needs of beginners and experts. The idea is that you should give users the simplest and most straightforward interface or experience possible when they first encounter a system or product, but then gradually reveal more complexity and advanced features as users become more familiar with the system. This can help prevent users from feeling overwhelmed or intimidated by a system that seems too complex, while still giving experienced users the tools they need to accomplish advanced tasks.

https://gpt-index.readthedocs.io/en/latest/guides/primer/query_interface.html

Querying an index or a graph involves a three main components:

Retrievers: A retriever class retrieves a set of Nodes from an index given a query.

Response Synthesizer: This class takes in a set of Nodes and synthesizes an answer given a query.

Query Engine: This class takes in a query and returns a Response object. It can make use of Retrievers and Response Synthesizer modules under the hood.

Chat Engine
https://gpt-index.readthedocs.io/en/latest/how_to/chat_engine/root.html

Chat engine is a high-level interface for having a conversation with your data (multiple back-and-forth instead of a single question & answer). Think ChatGPT, but augmented with your knowledge base.

Conceptually, it is a stateful analogy of a Query Engine. By keeping track of the conversation history, it can answer questions with past context in mind.

Guardrails is an open-source Python package for specification/validation/correction of output schemas. See below for a code example.
https://gpt-index.readthedocs.io/en/latest/how_to/output_parsing.html
https://pypi.org/project/guardrails/


Pentru cand vreau sa scot informatii cantitative din model
https://gpt-index.readthedocs.io/en/latest/examples/index_structs/struct_indices/SQLIndexDemo.html



https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html
Knowledge Graph Index



########### INSERTIE DOCUMENTE ############### 

In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import json
import os

import tiktoken
from openai.embeddings_utils import get_embedding
from sklearn.cluster import AgglomerativeClustering

import openai
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

# Read the ASIN values from the CSV file
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
#asin_list_path = './data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

OPENAI_API_KEY is ready


https://gpt-index.readthedocs.io/en/latest/how_to/index/usage_pattern.html#get-started

Read Data

In [5]:
from pathlib import Path
from llama_index import download_loader

SimpleCSVReader = download_loader("SimpleCSVReader")

loader = SimpleCSVReader()
reviews = loader.load_data(file=Path('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/reviews_export.csv'))
products = loader.load_data(file=Path('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/products_export.csv'))
weighted_graph = loader.load_data(file=Path('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/weighted_trait_graph_export.csv'))
cluster_solutions = loader.load_data(file=Path('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/cluster_solutions.csv'))

In [6]:
JSONReader = download_loader("JSONReader")

loader = JSONReader()
product_summary = loader.load_data(Path('/Users/vladbordei/Documents/Development/ProductExplorer/data/processed/summarised_product_information.json'))


Build Indices

In [7]:
reviews_index =  VectorStoreIndex.from_documents(reviews)
products_index = VectorStoreIndex.from_documents(products)
weighted_graph_index = VectorStoreIndex.from_documents(weighted_graph)
product_summary_index = VectorStoreIndex.from_documents(product_summary)
cluster_solutions_index = VectorStoreIndex.from_documents(cluster_solutions)

##### Build Query Engines

https://gpt-index.readthedocs.io/en/latest/examples/usecases/10q_sub_question.html

https://gpt-index.readthedocs.io/en/latest/guides/primer/usage_pattern.html

Right now, we support the following options:

default: “create and refine” an answer by sequentially going through each retrieved Node; This makes a separate LLM call per Node. Good for more detailed answers.

compact: “compact” the prompt during each LLM call by stuffing as many Node text chunks that can fit within the maximum prompt size. If there are too many chunks to stuff in one prompt, “create and refine” an answer by going through multiple prompts.

tree_summarize: Given a set of Node objects and the query, recursively construct a tree and return the root node as the response. Good for summarization purposes.

no_text: Only runs the retriever to fetch the nodes that would have been sent to the LLM, without actually sending them. Then can be inspected by checking response.source_nodes. The response object is covered in more detail in Section 5.

accumulate: Given a set of Node objects and the query, apply the query to each Node text chunk while accumulating the responses into an array. Returns a concatenated string of all responses. Good for when you need to run the same query separately against each text chunk.



index = ListIndex.from_documents(documents)

retriever = index.as_retriever()



query_engine = RetrieverQueryEngine.from_args(retriever, response_mode='default')



query_engine = RetrieverQueryEngine.from_args(retriever, response_mode='compact')



query_engine = RetrieverQueryEngine.from_args(retriever, response_mode='tree_summarize')



query_engine = RetrieverQueryEngine.from_args(retriever, response_mode='no_text')

In [55]:
reviews_query_engine = reviews_index.as_query_engine(similarity_top_k=5)
products_query_engine = products_index.as_query_engine(similarity_top_k=5)
weighted_graph_query_engine = weighted_graph_index.as_query_engine(similarity_top_k=5)
product_summary_query_engine = product_summary_index.as_query_engine(similarity_top_k=5)
cluster_solutions_query_engine = cluster_solutions_index.as_query_engine(similarity_top_k=5)

In [56]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=reviews_query_engine, 
        metadata=ToolMetadata(name='reviews', description='Product reviews')
    ),
    QueryEngineTool(
        query_engine=products_query_engine, 
        metadata=ToolMetadata(name='products', description='Provides information about products, as described on the Amazon pages')
    ),
    QueryEngineTool(
        query_engine=weighted_graph_query_engine, 
        metadata=ToolMetadata(name='aggregated reviews data', description='Aggregated information extracted from the reviews')
    ),
    QueryEngineTool(
        query_engine=product_summary_query_engine,
        metadata=ToolMetadata(name='product summary', description='A general description of the product')
    ),
    QueryEngineTool(
        query_engine=cluster_solutions_query_engine,
        metadata=ToolMetadata(name='cluster solutions', description='A database of solutions to problems that users have encountered')
    )
]

In [57]:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [58]:
response = s_engine.query('Analyze the issue with the product noise and let me know what good solutions are available.')

Generated 4 sub questions.
[reviews] Q: What is the issue with the product noise
[reviews] A: The issues with product noise include clicking sounds that can be annoying, loud and obnoxious volume, and loud and bothersome noise for plane travel. One review also mentions that the product can get annoying over time.
[products] Q: What is the product description
[products] A: The product description is not provided in the given context information.
[cluster solutions] Q: What are the common solutions to the issue with product noise
[cluster solutions] A: The common solutions to the issue with product noise are adding a volume control feature, incorporating noise-dampening materials like foam or rubber into the product design, and conducting a feasibility analysis for cost-effective implementation. These solutions should be user-friendly and highlighted in packaging and marketing materials, and the modifications should be tested to ensure they do not negatively impact the product's performa

In [59]:
print(response)

The issue with the product noise includes clicking sounds, loud and obnoxious volume, and bothersome noise for plane travel. One review also mentions that the product can get annoying over time. Common solutions to this issue are adding a volume control feature, incorporating noise-dampening materials like foam or rubber into the product design, and conducting a feasibility analysis for cost-effective implementation. Highly rated solutions include implementing noise reduction technology, including noise-cancelling headphones with the product, and providing a volume control feature. These solutions should be user-friendly and highlighted in packaging and marketing materials, and the modifications should be tested to ensure they do not negatively impact the product's performance or durability.


In [64]:
response = s_engine.query('what is the most frequently found issue with the product?')

Generated 5 sub questions.
[reviews] Q: What are the issues mentioned in the reviews
[reviews] A: The issues mentioned in the reviews include: 
- Loudness and rattling of a toy car 
- Beads getting stuck and decreased functionality over time in a Magnatab 
- Balls not staying up and getting stuck, as well as a detachable pen, in a toy for kids 
- Connectivity issues with a pen in a drawing and writing practice toy 
- Some magnets being harder to fit in a drawing toy 
- Stylus getting lost easily in a toy for young children 
- Some balls not working properly and a detachable pen in a toy for kids 
- Chips next to many holes and metal balls not staying up in those areas in a Magnatab.
[aggregated reviews data] Q: What is the frequency of each issue mentioned in the reviews
[aggregated reviews data] A: It is not possible to answer the question without further information. The context information provides examples of reviews mentioning various issues, but it does not provide a comprehensiv

In [65]:
print(response)

It is not possible to answer the question without further information. The context information provides examples of reviews mentioning various issues, but it does not provide a comprehensive list of all issues mentioned in the reviews. Additionally, the frequency of each issue would require analyzing the entire set of reviews, which is not provided in the context information.


In [66]:
response = s_engine.query('what is the most common use case?')

Generated 4 sub questions.
[product summary] Q: What are the different use cases of the product
[product summary] A: The product can be used for drawing, designing, creating, and playing with magnetic beads. It can also be used for teaching children how to write and draw, taking messages, completing classroom assignments, and practicing alphabets and numbers. It is suitable for use at home, schools, classrooms, long drives, flights, train rides, doctor's offices, waiting for a flight, restaurants, on-the-go, and travel. It promotes early writing education, STEM learning, and sensory-based learning. It encourages creativity, exploration, and problem-solving. It helps improve handwriting and fine motor skills. It is versatile and adaptable to different learning activities and environments.
[cluster solutions] Q: What are the common problems users have encountered
[cluster solutions] A: Users have encountered common problems such as beads getting stuck, difficulty in drawing, limited hole

In [67]:
print(response)

The most common use case of the product is not specified as multiple use cases are mentioned with equal emphasis.


In [ ]:
# Problema ramasa de rezolvat: trebuie introduse informatii cantitative in analiza: cat de comuna este o problema, etc. Toate informatiile primite pana acum au fost cantitative nu calitative.